In [ ]:
import featuretools as ft

es = ft.demo.load_mock_customer(return_entityset=True)

feature_defs = ft.dfs(
    entityset=es,
    target_entity="customers",
    agg_primitives=["mean"],
    trans_primitives=["time_since_previous"],
    features_only=True,
)

feature_defs

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_entity="customers",
    agg_primitives=["mean", "max", "min", "std", "skew"],
    trans_primitives=["time_since_previous"],
)

feature_matrix[[
    "MEAN(sessions.TIME_SINCE_PREVIOUS(session_start))",
    "MAX(sessions.TIME_SINCE_PREVIOUS(session_start))",
    "MIN(sessions.TIME_SINCE_PREVIOUS(session_start))",
    "STD(sessions.TIME_SINCE_PREVIOUS(session_start))",
    "SKEW(sessions.TIME_SINCE_PREVIOUS(session_start))",
]]

In [ ]:
ft.list_primitives().head(5)

In [ ]:
from featuretools.primitives import AggregationPrimitive, TransformPrimitive
from featuretools.tests.testing_utils import make_ecommerce_entityset
from featuretools.variable_types import Datetime, NaturalLanguage, Numeric
import pandas as pd

In [ ]:
class Absolute(TransformPrimitive):
    name = 'absolute'
    input_types = [Numeric]
    return_type = Numeric

    def get_function(self):
        def absolute(column):
            return abs(column)

        return absolute

In [ ]:
class Maximum(AggregationPrimitive):
    name = 'maximum'
    input_types = [Numeric]
    return_type = Numeric

    def get_function(self):
        def maximum(column):
            return max(column)

        return maximum

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_entity="sessions",
    agg_primitives=[Maximum],
    trans_primitives=[Absolute],
    max_depth=2,
)

feature_matrix.head(5)[[
    "customers.MAXIMUM(transactions.amount)",
    "MAXIMUM(transactions.ABSOLUTE(amount))",
]]

In [ ]:
class WordCount(TransformPrimitive):
    '''
    Counts the number of words in each row of the column. Returns a list
    of the counts for each row.
    '''
    name = 'word_count'
    input_types = [NaturalLanguage]
    return_type = Numeric

    def get_function(self):
        def word_count(column):
            word_counts = []
            for value in column:
                words = value.split(None)
                word_counts.append(len(words))
            return word_counts

        return word_count

In [ ]:
es = make_ecommerce_entityset()

feature_matrix, features = ft.dfs(
    entityset=es,
    target_entity="sessions",
    agg_primitives=["sum", "mean", "std"],
    trans_primitives=[WordCount],
)

feature_matrix[[
    "customers.WORD_COUNT(favorite_quote)",
    "STD(log.WORD_COUNT(comments))",
    "SUM(log.WORD_COUNT(comments))",
    "MEAN(log.WORD_COUNT(comments))",
]]

In [ ]:
class MeanSunday(AggregationPrimitive):
    '''
    Finds the mean of non-null values of a feature that occurred on Sundays
    '''
    name = 'mean_sunday'
    input_types = [Numeric, Datetime]
    return_type = Numeric

    def get_function(self):
        def mean_sunday(numeric, datetime):
            days = pd.DatetimeIndex(datetime).weekday.values
            df = pd.DataFrame({'numeric': numeric, 'time': days})
            return df[df['time'] == 6]['numeric'].mean()

        return mean_sunday

In [ ]:
feature_matrix, features = ft.dfs(
    entityset=es,
    target_entity="sessions",
    agg_primitives=[MeanSunday],
    trans_primitives=[],
    max_depth=1,
)

feature_matrix[[
    "MEAN_SUNDAY(log.value, datetime)",
    "MEAN_SUNDAY(log.value_2, datetime)",
]]